In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
from extractor.forex_extractor import FOREXExtractor
from time import sleep

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("prices")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2023-05-13 2023-05-20


In [4]:
market.connect()
data = FOREXExtractor.extract(start,end)
values = pd.DataFrame(data["rates"].values.tolist())
dates = data["rates"].keys().tolist()
values["dates"]= dates
market.store("forex",values)
market.disconnect()

In [5]:
values

,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,dates
0,3.672104,87.700535,101.421454,386.763563,1.796609,510.000367,228.263358,1.504438,1.80000,1.703970,...,2.70255,0.738882,599.625211,109.825037,250.350363,19.445495,9001.203589,18.267745,321.999592,2023-05-13
1,3.672075,87.700535,101.421454,386.763563,1.796609,509.999794,230.093250,1.504540,1.80000,1.691712,...,2.70255,0.738882,599.625211,109.825010,250.350134,19.190797,9001.198590,18.267745,321.999592,2023-05-14
2,3.672300,87.999437,102.297700,387.230355,1.802725,526.498706,230.641502,1.492315,1.80125,1.697133,...,2.70255,0.743704,602.999740,110.149988,250.350044,19.011898,9001.200129,18.705201,321.999592,2023-05-15
3,3.672100,87.999628,102.302742,386.350013,1.802270,527.501599,231.118599,1.502450,1.80000,1.706315,...,2.70255,0.743542,603.000021,110.249558,250.325017,19.084599,9001.196392,18.699978,321.999592,2023-05-16
4,3.672098,87.502134,104.498212,386.309619,1.801776,541.496182,231.622897,1.502570,1.80000,1.694587,...,2.70255,0.743363,604.500999,110.503157,250.302706,19.268297,9001.208525,18.570690,321.999592,2023-05-17
5,3.671799,87.503383,103.690112,387.029743,1.802661,540.497441,232.134704,1.508750,1.80250,1.700085,...,2.70255,0.743687,604.513532,110.899792,250.350237,19.338850,9001.200260,18.654767,321.999592,2023-05-18
6,3.672304,87.000368,103.725041,386.420403,1.802364,540.503981,232.405507,1.503533,1.80200,1.703970,...,2.70255,0.743596,604.503597,110.903595,250.350363,19.450880,9001.203589,18.902117,321.999592,2023-05-19
7,3.672304,87.000368,103.725041,386.420403,1.802364,540.503981,232.405507,1.503533,1.80200,1.703970,...,2.70255,0.743596,604.503597,110.903595,250.350363,19.450880,9001.203589,18.902117,321.999592,2023-05-20


In [6]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("prices",data)
        except:
            print(ticker,"tiingo")
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

  1%|▎                                          | 4/490 [00:06<13:07,  1.62s/it]

market prices documents must be a non-empty list


 73%|█████████████████████████████▊           | 357/490 [09:36<03:35,  1.62s/it]

market prices documents must be a non-empty list


 90%|████████████████████████████████████▊    | 440/490 [11:50<01:20,  1.61s/it]

market prices documents must be a non-empty list


100%|█████████████████████████████████████████| 490/490 [13:11<00:00,  1.62s/it]


In [7]:
data

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,ticker
0,2023-05-15T00:00:00.000Z,183.85,184.3200,182.72,183.60,1060570,183.85,184.3200,182.72,183.60,1060570,0.0,1.0,ZTS
1,2023-05-16T00:00:00.000Z,181.45,183.6200,180.84,182.87,1175273,181.45,183.6200,180.84,182.87,1175273,0.0,1.0,ZTS
2,2023-05-17T00:00:00.000Z,178.94,182.4250,177.72,182.35,1714474,178.94,182.4250,177.72,182.35,1714474,0.0,1.0,ZTS
3,2023-05-18T00:00:00.000Z,179.72,180.1700,175.86,178.92,1836587,179.72,180.1700,175.86,178.92,1836587,0.0,1.0,ZTS
4,2023-05-19T00:00:00.000Z,179.55,182.1404,179.19,180.83,1536946,179.55,182.1404,179.19,180.83,1536946,0.0,1.0,ZTS


In [8]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|█████████████████████████████████████████| 490/490 [00:33<00:00, 14.78it/s]


In [9]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
88,-0.182178,-0.409250,-0.258380,0.119916,0.156503,CTLT,Catalent,Health Care,Pharmaceuticals
170,-0.253213,-0.167283,-0.112935,0.045627,0.039392,EPAM,EPAM Systems,Information Technology,IT Consulting & Other Services
8,0.101894,-0.023232,-0.007751,0.074001,0.030020,ADBE,Adobe Inc.,Information Technology,Application Software
71,0.149889,-0.052361,-0.034721,0.036147,0.026012,BWA,BorgWarner,Consumer Discretionary,Auto Parts & Equipment
157,-0.116733,-0.035136,0.023879,0.047578,0.025168,DXC,DXC Technology,Information Technology,IT Consulting & Other Services
362,-0.026123,-0.020762,-0.037098,0.010446,0.024366,PXD,Pioneer Natural Resources,Energy,Oil & Gas Exploration & Production
169,-0.057865,-0.049754,-0.034137,0.029190,0.022942,EOG,EOG Resources,Energy,Oil & Gas Exploration & Production
197,0.435078,0.050068,0.108581,0.015607,0.021867,FTNT,Fortinet,Information Technology,Systems Software
43,-0.224713,-0.142052,-0.094580,0.014268,0.021400,APA,APA Corporation,Energy,Oil & Gas Exploration & Production
250,-0.021682,-0.002451,0.049684,0.041625,0.020414,IQV,IQVIA,Health Care,Life Sciences Tools & Services


In [10]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Consumer Staples,0.030854,0.009626,-0.029330,-0.009914,0.001692
Consumer Discretionary,0.088449,-0.004490,-0.015247,0.000434,-0.014602
Health Care,0.003260,-0.005180,-0.033226,-0.005638,0.003958
Industrials,0.051388,-0.006107,-0.003880,0.007657,-0.004321
Information Technology,0.121314,-0.013436,0.022435,0.026429,-0.002101
Communication Services,0.073485,-0.014724,-0.001327,0.009848,-0.010744
Utilities,-0.045681,-0.021493,-0.046555,-0.028851,-0.000816
Financials,-0.086173,-0.025808,-0.025254,0.019152,-0.004844
Real Estate,-0.040780,-0.025864,-0.040762,-0.020472,-0.003642
